In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"mrleo087","key":"37c58ffd7b80bb7ac27fc9a2adb6522b"}'}

In [3]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json


In [4]:
!kaggle datasets download -d 'purusinghvi/email-spam-classification-dataset'

Dataset URL: https://www.kaggle.com/datasets/purusinghvi/email-spam-classification-dataset
License(s): MIT
email-spam-classification-dataset.zip: Skipping, found more recently modified local copy (use --force to force download)


In [5]:
import zipfile
with zipfile.ZipFile('/content/email-spam-classification-dataset.zip','r') as zip_ref:
  zip_ref.extractall('/content/dataset')

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings('ignore')

In [7]:
df = pd.read_csv('/content/dataset/combined_data.csv')

In [8]:
df

,label,text
0,1,ounce feather bowl hummingbird opec moment ala...
1,1,wulvob get your medircations online qnb ikud v...
2,0,computer connection from cnn com wednesday es...
3,1,university degree obtain a prosperous future m...
4,0,thanks for all your answers guys i know i shou...
...,...,...
83443,0,hi given a date how do i get the last date of ...
83444,1,now you can order software on cd or download i...
83445,1,dear valued member canadianpharmacy provides a...
83446,0,subscribe change profile contact us long term ...


In [9]:
df['label'].value_counts()

,count
label,
1,43910
0,39538


In [10]:
df.shape

(83448, 2)

In [11]:
X = df['text']
y = df['label']

In [12]:
X = X.str.lower()

In [13]:
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer


In [14]:
tokenizer = Tokenizer(oov_token='<OOV>')
tokenizer.fit_on_texts(X)

In [15]:
vocal_size = len(tokenizer.word_index)+1

In [16]:
X_seq = tokenizer.texts_to_sequences(X)

In [17]:
max_len = max(len(x) for x in X_seq)

In [18]:
max_len

101984

In [19]:
len_seq = [len(x) for x in X_seq]
count = 0
for i in len_seq:
  if i > 700:
    count+=1

count

4566

In [20]:
max_len = 700

In [21]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
X_padded = pad_sequences(X_seq,maxlen=max_len,padding='post',truncating = 'post')

In [22]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Embedding,GRU,Dense,Dropout,Input

In [23]:
inputs = Input(shape=(max_len,))
x = Embedding(input_dim=vocal_size,output_dim=200)(inputs)
x = GRU(150,return_sequences=True)(x)
x = Dropout(0.3)(x)

x = GRU(125)(x)
x = Dropout(0.2)(x)

outputs = Dense(1,activation='sigmoid')(x)

model = Model(inputs = inputs, outputs = outputs)


In [24]:
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 700)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ embedding (Embedding)           │ (None, 700, 200)       │    64,717,200 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 700, 150)       │       158,400 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 700, 150)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 125)            │       103,875 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 125)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 1)              │           126 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 64,979,601 (247.88 MB)

 Trainable params: 64,979,601 (247.88 MB)

 Non-trainable params: 0 (0.00 B)

In [25]:
model.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [26]:
model.fit(X_padded,y,epochs=3,validation_split=0.2)

Epoch 1/3
2087/2087 ━━━━━━━━━━━━━━━━━━━━ 190s 88ms/step - accuracy: 0.5622 - loss: 0.6575 - val_accuracy: 0.9868 - val_loss: 0.0434
Epoch 2/3
2087/2087 ━━━━━━━━━━━━━━━━━━━━ 196s 94ms/step - accuracy: 0.9901 - loss: 0.0328 - val_accuracy: 0.9923 - val_loss: 0.0261
Epoch 3/3
2087/2087 ━━━━━━━━━━━━━━━━━━━━ 186s 89ms/step - accuracy: 0.9983 - loss: 0.0067 - val_accuracy: 0.9913 - val_loss: 0.0355


In [27]:
model.save('email_rnn.keras')

In [52]:
test =  ["""
 Thank you for registering for the upcoming Data Science and Machine Learning Workshop scheduled for next month. This email serves as confirmation of your successful registration and provides additional details to help you prepare.

The workshop will focus on practical applications of machine learning, including data preprocessing, model selection, and performance evaluation. Sessions will be conducted by experienced industry professionals and will include hands-on exercises designed to reinforce key concepts.

A detailed agenda, along with preparatory materials, will be shared one week prior to the event. We recommend reviewing the materials in advance to maximize your learning experience. Participants are encouraged to bring their own laptops with the required software installed.

If you have any dietary restrictions or accessibility requirements, please inform us by replying to this email no later than Friday. Our team is committed to ensuring a comfortable and inclusive environment for all attendees.

Should you have any questions regarding the schedule, location, or content, feel free to contact the organizing committee. We look forward to your active participation and hope you find the workshop both informative and engaging.
"""]

test_seq =tokenizer.texts_to_sequences(test)
test_padded = pad_sequences(test_seq,maxlen=max_len,padding='post',truncating = 'post')
y_pred = model.predict(test_padded)
if y_pred>=0.5:
  print(f"SPAM : {y_pred}")

else:
  print(f"NOT SPAM : {y_pred}")




1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
NOT SPAM : [[0.00015949]]


In [37]:
import pickle
with open('tokenizer_email.pkl','wb') as f:
  pickle.dump(tokenizer,f)